# Deploying models with local dependencies on Verta

Within Verta, a "Model" can be any arbitrary function: a traditional ML model (e.g., sklearn, PyTorch, TF, etc); a function (e.g., squaring a number, making a DB function etc.); or a mixture of the above (e.g., pre-processing code, a DB call, and then a model application.) See more [here](https://docs.verta.ai/verta/registry/concepts).

This notebook provides an example of how to deploy a PyTorch model with local dependencies on Verta.

This example features:
- **PyTorch** fully-connected neural network implemented in a separate module (`models/nets.py`)
- **verta**'s Python client logging training results
- **verta**'s Python client logging user-defined modules (`models/nets.py` and `utils/preprocess.py`) for deployment

## 0. Imports

In [1]:
from __future__ import print_function

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import itertools
import os
import time

import six

import numpy as np
import pandas as pd

import torch
import torch.optim as optim
import torch.utils.data as data_utils

### 0.1 Verta import and setup

In [2]:
try:
    import verta
except ImportError:
    !pip install verta

In [3]:
import os

# Ensure credentials are set up, if not, use below
# os.environ['VERTA_EMAIL'] = 
# os.environ['VERTA_DEV_KEY'] = 
# os.environ['VERTA_HOST'] =

from verta import Client

client = Client(os.environ['VERTA_HOST'])

---

## 1. Model Training

### 1.1 Load Training data

In [4]:
from utils.data import load_mnist

X, y = load_mnist()

In [5]:
# gather indices to split training data into training and validation sets
shuffled_idxs = np.random.permutation(len(y))
idxs_train = shuffled_idxs[int(len(shuffled_idxs)/10):]  # last 90%
idxs_val = shuffled_idxs[:int(len(shuffled_idxs)/10)]  # first 10%

X_train, y_train = X[idxs_train], y[idxs_train]
X_val, y_val = X[idxs_val], y[idxs_val]

In [6]:
# create Dataset object to support batch training
class TrainingDataset(data_utils.Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return (self.features[idx], self.labels[idx])

### 1.2 Define Model

In [7]:
from models.nets import FullyConnected

hidden_size = 512
dropout = 0.2
model = FullyConnected(num_features=X.shape[1], hidden_size=hidden_size, dropout=dropout)

In [8]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
num_epochs = 5
batch_size = 32

### 1.3 Train model

In [9]:
# enable batching of training data
dataset = TrainingDataset(X_train, y_train)
dataloader = data_utils.DataLoader(dataset,
                                   batch_size=batch_size,
                                   shuffle=True)

In [10]:
for i_epoch in range(num_epochs):
    for i_batch, (X_batch, y_batch) in enumerate(dataloader):
        model.zero_grad()  # reset model gradients

        output = model(X_batch)  # conduct forward pass

        loss = criterion(output, y_batch)  # compare model output w/ ground truth
        
        print("\repoch {}/{} | ".format(i_epoch+1, num_epochs), end='')
        print("iteration {}/{} | ".format(i_batch+1, len(dataloader)), end='')
        print("epoch loss avg: {}".format(loss.item()), end='')

        loss.backward()  # backpropogate loss to calculate gradients
        optimizer.step()  # update model weights
    with torch.no_grad():  # no need to calculate gradients when assessing accuracy
        print()
        
        pred_train = model(X_train).numpy().argmax(axis=1)
        train_acc = (pred_train == y_train.numpy()).mean()
        print("Training accuracy: {}".format(train_acc))
        
        pred_val = model(X_val).numpy().argmax(axis=1)
        val_acc = (pred_val == y_val.numpy()).mean()
        print("Validation accuracy: {}".format(val_acc))

## Calculate and Log Accuracy on Full Training Set

In [11]:
with torch.no_grad():  # no need to calculate gradients when assessing accuracy
    pred_train = model(X_train).numpy().argmax(axis=1)
    train_acc = (pred_train == y_train.numpy()).mean()
    print("Training accuracy: {}".format(train_acc))

In [12]:
torch.save(model, "model.pth")
print("Saved PyTorch Model State to model.pth")

In [13]:
from verta.registry import VertaModelBase

class MNISTClassifier(VertaModelBase):
    def __init__(self, artifacts):
        from models.nets import FullyConnected
        self.model = torch.load(artifacts["model.pth"])
        self.model.eval()
        
    def predict(self, batch_input):
        as_tensor = torch.FloatTensor(batch_input)
        return self.model(as_tensor).detach().numpy()

In [14]:
mnist_clf = MNISTClassifier(artifacts={"model.pth" : "model.pth"})
mnist_clf.predict(X.tolist())

---

## 2. Register Model for deployment

In [15]:
registered_model = client.get_or_create_registered_model(
    name="mnist-custom-modules", labels=["computer-vision", "pytorch"])

### `utils/preprocess.py` is used in `models/nets.py`, so it must be supplied as custom modules

In [16]:
from verta.environment import Python

model_version = registered_model.create_standard_model(
    model_cls=MNISTClassifier,
    environment=Python(requirements=["torch"]),
    artifacts={"model.pth" : "model.pth"},
    name="v-2",
    code_dependencies=["models/nets.py", "utils/preprocess.py"]
)

---

## 3. Deploy model to endpoint

In [17]:
mnist_endpoint = client.get_or_create_endpoint("mnist-custom-modules")
mnist_endpoint.update(model_version, wait=True)

In [18]:
deployed_model = mnist_endpoint.get_deployed_model()
deployed_model.predict(X)

---